------------------------------------------------------------------------

Copyright 2023 Christopher Fazekas [Alejandro Chavez Lab at UCSD]

All Rights Reserved

MAVDA Academic License

preprocess.ipynb

------------------------------------------------------------------------

In [ ]:
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pandas as pd
import numpy as np
import os
import glob
import gzip
pd.options.mode.chained_assignment = None

constants - 1. a merged.csv file, containing indexing information for all possible primer plates

            ,Well,7-series_ID,5-series_ID,7-series_SEQ,5-series_SEQ,Internal_ID,Internal_Seq,PP,SAMPLE_ID,merged_index
            0,A1,n701,s502,TAAGGCGA,ATAGAGAG,1,AATGC,1,A170,TAAGGCGAATAGAGAGAATGC
            1,A2,n701,s502,TAAGGCGA,ATAGAGAG,2,GTTGC,1,A170,TAAGGCGAATAGAGAGGTTGC
            2,A3,n701,s502,TAAGGCGA,ATAGAGAG,3,GATCA,1,A170,TAAGGCGAATAGAGAGGATCA
            3,A4,n701,s502,TAAGGCGA,ATAGAGAG,4,TGATC,1,A170,TAAGGCGAATAGAGAGTGATC
            4,A5,n701,s502,TAAGGCGA,ATAGAGAG,5,ACTCC,1,A170,TAAGGCGAATAGAGAGACTCC
            5,A6,n701,s502,TAAGGCGA,ATAGAGAG,6,ACCTG,1,A170,TAAGGCGAATAGAGAGACCTG   


input - 1. database of fastq.gz files, already aligned to illuminia indicies
        get indexes (i7,i5) as well as internal barcode index and model index
        which are both at a fixed position in the read 

        2. A setup csv file, containing the plate ID (names) and associated primer 
        plate. 

        plateID,primerplate
        NS1519_22057,1
        NS1519_22058,13
        NS1519_22059,7
        NS1519_22060,10

        3. tbd format of OD file (??? not required for counts, maybe for backend)

output - 
        1. counts table of the format:

                    wellid, wellid, wellid, wellid, wellid  x96
            model       25      7       29      30      20
            model       23      2       23      15      10
            model       12      4       12      35      15
            x217

for each platename and primer plate number in the setup csv, pull corresponding .gz files into 
a separate working folder. 

generate a flat dictionary (countsdict) where {key:value} == {wellid_model:count}
i.e {GGACTCCTAGAGGATACGATA:1}

for each gz in the working folder
    set i7 and i5 indicies
    skip to 2nd line, get column id and protease id
    set wellid_model 
    countsdict[wellid_model] += 1

convert the dictionary to an output plateid_counts.csv

In [ ]:
setupname = "setup.csv"



direc = os.getcwd()
setuppath = direc + "/" + setupname
mergedpath = direc + "/" + "merged.csv"
models = direc + "/" + "models.csv"

#reading in runinfo - plateIDs and assocaited primer plate IDs 
r = pd.read_csv(setuppath)
runinfo = r.set_index("plateID").to_dict()

#reading in merged dataframe
m = pd.read_csv(mergedpath, index_col=0)

#reading in models
g = pd.read_csv(models, index_col=0)

def returnrunindices(merged,primerplatenum):
    #takes merged primer plate frame and primer plate number and returns a dataframe
    #containing only the indices from the primer plate
    returndf = merged[merged["PP"]==primerplatenum]
    return returndf

def modelwiseindicies(runindicies, modelcsv):
    #takes the well incidies for a plate (or plates) and returns a dataframe containing the model-wise
    #index for each well
    modellst = list(modelcsv["model"] + "_" + modelcsv["seq"])
    runindicies["model_seq"] = runindicies.apply(lambda x: modellst, axis=1)
    retdf = runindicies.explode("model_seq")
    retdf["fullseq"] = retdf["merged_index"].str[:] + retdf["model_seq"].str[-10:]
    return retdf

print(r)


In [ ]:
for plate in runinfo['primerplate'].items():
    plateID = plate[0]
    ppID = plate[1]
    indexsub = m[m["PP"]==ppID] #get subset of indicies for all models all wells in a plate
    subind = modelwiseindicies(indexsub, g)
    counts = dict((el,0) for el in subind["fullseq"]) #form counts dictionary for each full index sequence
    asers = subind["SAMPLE_ID"].unique() #get associated a-series indicies from the primer plate (lst)
    for a in asers:
        rowsubind = subind[subind["SAMPLE_ID"]==a]
        sevenser = rowsubind["7-series_SEQ"].unique()[0] #grab row 7-series seq
        fiveser = rowsubind["5-series_SEQ"].unique()[0] #grab row 5-series seq
        for parta in glob.iglob(direc + "/gz/" + a + "*R1_001.fastq.gz"): #iterator for all .fastq.gz files of type a
            readstats = [0,0] #unindexed counts, total counts
            with gzip.open(parta, "rt") as handle: #open .gz
                for title, sequence, quality in FastqGeneralIterator(handle): #iterator for fastq files (returns tup(title,sequence,quality))
                    countsind = sevenser + fiveser + sequence[4:9] + sequence[45:55] #form index seqeunce
                    readstats[1] += 1
                    try:
                        counts[countsind] += 1 #increment sequence count by 1
                    except:
                        readstats[0] += 1
                        pass
                print(parta + " total counts:" + str(readstats[1]) + "; unaligned:" + str(readstats[0]) + "; " + str(100 - round((readstats[0]/readstats[1])*100,2)) + "%" +" alignment")
    subind["counts"] = subind["fullseq"].map(counts) #map counts dictionary to subind dataframe
    totable = subind[["Well","model_seq","counts"]] #create output df from subind, reorganize order, names of columns, set barcode and well, ect. 
    totable["runid"] = plateID
    totable["gene"] = totable["model_seq"].str.split("_").str[0]
    totable["barcode"] = totable["model_seq"].str.split("_").str[1]
    totable = totable.drop(["model_seq"], axis=1)
    totable = totable.rename(columns={"Well":"well"})
    finorder = ["runid","well","gene","barcode","counts"]
    totable = totable.reindex(columns=finorder)
    os.makedirs(direc + "/counttables", exist_ok=True)
    totable.to_csv(direc + "/counttables/" + plateID + "_" + "PP" + str(ppID) + ".csv", index=False) #output counts csv
